<a href="https://colab.research.google.com/github/egorchistov/SfM/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo: Scale trajectory predicted by [SfM Learner](https://github.com/ClementPinard/SfmLearner-Pytorch)

Download example sequences including KITTI Odometry 09-10 and two sequences from our phones

In [ ]:
%%shell
gdown --fuzzy https://drive.google.com/file/d/14HTQ0-6MVzrQ9MIr9b7Ym5TJMUCsolXw/view?usp=sharing -O odometry.zip
unzip odometry.zip > /dev/null

Clone SfM Learner and download pretrained weights

In [ ]:
%%shell
git clone https://github.com/ClementPinard/SfmLearner-Pytorch
cd SfmLearner-Pytorch
pip install -qr requirements.txt
mkdir -p pretrained
gdown --fuzzy https://drive.google.com/file/d/1o-cKKbq8jGvjaiEKuKWDjEMmI5Y4PN9K/view?usp=sharing -O pretrained/exp_pose_model_best.pth.tar
gdown --fuzzy https://drive.google.com/file/d/1SOyTw_XDzirdf-tSz_TV7HjAG4wWB3nr/view?usp=sharing -O pretrained/dispnet_model_best.pth.tar

Clone our repository with helper functions

In [ ]:
%%shell
git clone https://github.com/egorchistov/SfM
cd SfM
pip install -qr requirements.txt

Patch dataloader to use .jpg files instead of .png files

In [ ]:
%%shell
cd SfmLearner-Pytorch
git apply <<EOF
diff --git a/kitti_eval/pose_evaluation_utils.py b/kitti_eval/pose_evaluation_utils.py
index 3db48c0..1b31c3d 100644
--- a/kitti_eval/pose_evaluation_utils.py
+++ b/kitti_eval/pose_evaluation_utils.py
@@ -50,11 +50,11 @@ def read_scene_data(data_root, sequence_set, seq_length=3, step=1):
     print('getting test metadata for theses sequences : {}'.format(sequences))
     for sequence in tqdm(sequences):
         poses = np.genfromtxt(data_root/'poses'/'{}.txt'.format(sequence.name)).astype(np.float64).reshape(-1, 3, 4)
-        imgs = sorted((sequence/'image_2').files('*.png'))
+        imgs = sorted((sequence/'image_2').files('*.jpg'))
         # construct 5-snippet sequences
         tgt_indices = np.arange(demi_length, len(imgs) - demi_length).reshape(-1, 1)
         snippet_indices = shift_range + tgt_indices
         im_sequences.append(imgs)
         poses_sequences.append(poses)
         indices_sequences.append(snippet_indices)
-    return im_sequences, poses_sequences, indices_sequences
\ No newline at end of file
+    return im_sequences, poses_sequences, indices_sequences
EOF

Patch run_inference.py to correctly load images

In [ ]:
%%shell
cd SfmLearner-Pytorch
git apply <<EOF
diff --git a/run_inference.py b/run_inference.py
index 2bf74d1..42f031a 100644
--- a/run_inference.py
+++ b/run_inference.py
@@ -60,6 +60,9 @@ def main():
         h,w,_ = img.shape
         if (not args.no_resize) and (h != args.img_height or w != args.img_width):
             img = resize(img, (args.img_height, args.img_width))
+        else:
+            img = img / 255
+
         img = np.transpose(img, (2, 0, 1))
 
         tensor_img = torch.from_numpy(img.astype(np.float32)).unsqueeze(0)
EOF

Predict poses for some sequences here

In [ ]:
%%shell
cd SfmLearner-Pytorch
for sequence in "kitti_odometry_09" # "kitti_odometry_10" "phone_1" "phone_2"
do
    python test_pose.py                                \
        pretrained/exp_pose_model_best.pth.tar         \
        --dataset-dir "../odometry"                    \
        --sequences "$sequence"                        \
        --output-dir "../odometry/sequences/$sequence" \
        --img-width 416                                \
        --img-height 128                               \
        --no-resize
done

Predict disparity for some sequences here

In [ ]:
%%shell
cd SfmLearner-Pytorch
for sequence in "kitti_odometry_09" # "kitti_odometry_10" "phone_1" "phone_2"
do
    mkdir -p "../odometry/sequences/$sequence/disparity"
    python run_inference.py                                      \
        --pretrained pretrained/dispnet_model_best.pth.tar       \
        --dataset-dir "../odometry/sequences/$sequence/image_2"  \
        --output-dir "../odometry/sequences/$sequence/disparity" \
        --output-disp                                            \
        --img-width 416                                          \
        --img-height 128                                         \
        --no-resize
done

Find trajectory scale

In [ ]:
%%shell
cd SfM
for sequence in "kitti_odometry_09" # "kitti_odometry_10" "phone_1" "phone_2"
do
    python scale.py \
        --sequence "../odometry/sequences/$sequence"
done

Prepare trajectories for visualization

In [ ]:
%%shell
cd SfM
for sequence in "kitti_odometry_09" # "kitti_odometry_10" "phone_1" "phone_2"
do
    python visualize.py         \
        --dataset "../odometry" \
        --sequence "$sequence"
done

Visualize trajectories

In [ ]:
%%shell
pip install -q tornado~=5.1.0

In [ ]:
import pandas as pd
import plotly.express as px

for sequence in [
        "kitti_odometry_09",
#         "kitti_odometry_10",
#         "phone_1",
#         "phone_2"
]:
    df = pd.read_csv(f"odometry/sequences/{sequence}/visualize.csv")
    fig = px.scatter(df, x="x", y="y", z="z", color="label", hover_data=["length", "frame"])
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.show()

To use with your own data
* Put .jpg files in `odometry/sequences/my/image_2`
* Find camera intrinsics and save them to `odometry/sequences/my/cam.txt`
* Save camera height to `odometry/sequences/my/height.txt`
* Generate placeholder instead of ground truth poses and place it in `odometry/poses/my.txt`